# 3 - Other endpoints

There are further endpoints available on Discovery, with more specific functionality. We will look at some basic useage examples of the `children` and `context` endpoints here, and provide starting information on the final ones for you to explore.

As always, we start by installing and importing the libraries we need.

In [ ]:
%pip install -q requests
%pip install -q json
import requests
import json

## 3.1 Working through collections

Records are typically arragned into collections, and Discovery has two endpoints to help you navigate through them, depending on whether you want to climb or descend the hierarchy.

From [notebook 2](2-working-through-results.ipynb) we gathered the ID of a record and of its parent, which we will use here. 

In [ ]:
base_discovery_url = "https://discovery.nationalarchives.gov.uk/API"

In [ ]:
parent_id = "F236668"

In [ ]:
record_id = {harshad, fill this in}

### 3.1.1 - Children endpoint

The first new endpoint we are going to use is `/records/children/{parentId}`. The parent ID here is case-sensitive, so if we use `f`236668 instead of `F`236668 then it won't return any result.

This endpoint lets you get all children of a record, based on its ID. Combined with the previous ID, you can get all siblings of a record, thus the whole collection.

In [ ]:
collection_endpoint = "/records/children/"

In [ ]:
full_search_url = base_discovery_url + collection_endpoint + parent_id

response = requests.request("GET", full_search_url)

print(json.dumps(response.json(), indent=4))

From this first test run, we can check how many results were returned initially, and whether there are more results available. 

In [ ]:
print(len(response.json()['assets']), response.json()['hasMoreAfterLast'])

As we know there are more results available, we will start by saving this first set of results as a new list.

In [ ]:
sibling_records = response.json()['assets']

We can then set up a loop to get the rest of the results, checking to see if `hasMoreAfterLast` is true. If it is, we will use the ID of the last record in the list to get the next set of results. When `hasMoreAfterLast` is false we know we have all the results.

In [ ]:
while(response.json()['hasMoreAfterLast']):
  last_id = response.json()['assets'][-1]["id"]
  full_search_url = base_discovery_url + collection_endpoint + parent_id + "?direction=NEXT&batchStartRecordId=" + last_id
  response = requests.request("GET", full_search_url)
  sibling_records = sibling_records + response.json()['assets'][1:]
print(len(sibling_records))

With different levels of hierarchy in different record series, it can also be worth using this endpoint to check whether there are any further children of any children that have been found. Here, we will just check the first child of the first record in the list, but this could be incorporated into the loop above recursively.

In [ ]:
parent_id =  "N14309820" # TODO: Harshad, confirm this is a child of the parent in the earlier cell and make it clear if it is

In [ ]:
full_search_url = base_discovery_url + collection_endpoint + parent_id

response = requests.request("GET", full_search_url)

print(json.dumps(response.json(), indent=4))

In this case, there are no further children, so we can move on to the next endpoint.

### 3.1.2 - Context endpoint

The context endpoint is roughly the opposite of the children endpoint. Providing and ID will return the entire hierarchy above it, up to and including the holding information's ARCHON data. The returned data are similarly structured to the children endpoint. Note: this endpoint is a bit slow, but can proivide a lot of data in a single call - consider if it is the best option for your use case before using it.

In [ ]:
context_endpoint = "/records/context/"

context_full_search_url = base_discovery_url + context_endpoint + record_id

response = requests.request("GET", context_full_search_url)

print(json.dumps(response.json(), indent=4))

# 3.2 - The other endpoints

At this stage, we have still note covered all the endpoints available on Discovery, but to avoid many more notebooks, we are going to finish here with just a brief look at the others. 

They all work in a similar way to the endpoints we have already covered, with the documentation provided in the Sandbox. With some adjustments to query parameters and endpoints, the code and logic in these notebooks can be used to explore them.

The endpoints are grouped by general area of work: 
`SearchForRepositories` and `Repository` are used to find archives and repositories, returning data such as physical addresses of the organisation, or holding information on who holds the repository.
`SearchForFileAuthorities` and `FileAuthority` are used to find information on {I don’t know}.


# 4 - Conclusion

Discovery is a rich and varied database, providing access to TNAs rich and diverse record collection. Fittingly, there are a range of different endpoints made available to answer the many different research questions that may be asked. 

Hopefully these notebooks have helped introduce Discovery API, and shown how to gather precise and accurate data efficiently and effectively, and given you the confidence to use it in your future research projects. 

The next section of notebooks are some case studies, giving examples of what you can do with the data you’ve gathered so far. 
